In [95]:
from mods import readdata as rd
from mods import savedata as sd
from mods import extractdata as ed
from mods import gmap as gm
from mods import connectDB as connDB
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd
import pymysql

In [2]:
load_dotenv()

True

In [53]:
# 讀取raw data
path = "data/raw/pet_establishment.csv"
df = rd.get_csv_data(path)
df

[✓] CSV檔案已取回



,ID,legaltype,legalname,legaladdress,busitem,animaltype,validnum,validdate,own_name,bos_name,rank_year,rank_code,rank_flag_1,rank_flag_2,rank_text,state_flag
0,15913,A010,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,特寵業字第V1120348號,2027/11/17 上午 12:00:00,宣家瑩,許玲玲,2024.0,B,Y,N,NaN,N
1,19029,A090,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,特寵業繁字第C1140733號,2028/10/2 上午 12:00:00,黃韻蓁,袁子榕,NaN,NaN,NaN,NaN,NaN,N
2,19023,A210,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,特寵業繁字第U1140846號,2028/10/7 上午 12:00:00,施寶美,劉品艾,NaN,NaN,NaN,NaN,NaN,N
3,17813,A060,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,特寵業繁字第S1130016號,2027/3/14 上午 12:00:00,許小芸,許芸鳳、黃信捷,NaN,NaN,NaN,NaN,NaN,N
4,19041,A020,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,特寵業繁字第A1140069號,2028/10/19 上午 12:00:00,洪蔡佳芬,周宜潔,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,17,A040,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,特寵業繁字第W1080095號,2022/1/10 上午 12:00:00,張志雄,彭秀惠,2019.0,B,Y,N,NaN,B
5500,6,A040,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,特寵業繁字第W1080128號,2022/3/14 上午 12:00:00,詹東旺,曾健倫,2019.0,B,Y,N,NaN,P
5501,4,A040,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,特寵業繁字第W1070135號,2021/11/16 上午 12:00:00,周張月琴,周進欽,2019.0,B,Y,N,NaN,P
5502,3,A040,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,特寵業字第Ｗ1070144號,2024/9/25 上午 12:00:00,吳慧珠,龔高益,2019.0,C,Y,N,NaN,P


In [54]:
# 執行ETL
# 去除不必要的欄位
df_filtered = df[["legalname", "legaladdress", "busitem", "animaltype", "state_flag"]]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
1,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,N
2,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N
3,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N
4,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N
...,...,...,...,...,...
5499,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,B
5500,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,P
5501,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,P
5502,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,P


In [55]:
# busitem = C表示寄養，state_flag = N表示營業中
df_filtered = df_filtered[
    (~df_filtered["busitem"].str.contains("A", na=False))
    & (df_filtered["busitem"].str.contains("C", na=False))
    & (df_filtered["state_flag"] == "N")
]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
5,東森寵物雲股份有限公司花蓮玉里分公司,花蓮縣玉里鎮和平路96號,C,狗、貓,N
7,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,BC,狗、貓,N
8,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,BC,狗、貓,N
9,樂沛寵物,高雄市楠梓區土庫一路10號,C,狗、貓,N
...,...,...,...,...,...
5242,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,BC,狗、貓,N
5274,編緣貓企業社,臺中市南區南門路47號1樓,C,貓,N
5337,貓的旅坊,臺南市東區裕農路691號1-2樓,C,貓,N
5420,貓屋住宿用品店,雲林縣斗六市永安路77號,C,貓,N


In [56]:
# 找出屬於六都的機構
df_filtered["city"], df_filtered["district"] = zip(
    *df_filtered["legaladdress"].apply(ed.extract_city_district)
)
df_filtered = df_filtered[df_filtered["city"].notna()].reset_index(drop=True)
df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_3904\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(
C:\Users\dinni\AppData\Local\Temp\ipykernel_3904\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(


,legalname,legaladdress,busitem,animaltype,state_flag,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N,臺北市,大安區
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,BC,狗、貓,N,高雄市,左營區
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,BC,狗、貓,N,高雄市,楠梓區
3,樂沛寵物,高雄市楠梓區土庫一路10號,C,狗、貓,N,高雄市,楠梓區
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,C,狗,N,臺北市,萬華區
...,...,...,...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,BC,狗、貓,N,臺中市,南區
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,BC,狗、貓,N,桃園市,龜山區
1266,編緣貓企業社,臺中市南區南門路47號1樓,C,貓,N,臺中市,南區
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,C,貓,N,臺南市,東區


In [57]:
# 只留下名稱、地址、市和區
df_filtered = df_filtered[["legalname", "legaladdress", "city", "district"]]
df_filtered

,legalname,legaladdress,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,臺北市,大安區
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,高雄市,左營區
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,高雄市,楠梓區
3,樂沛寵物,高雄市楠梓區土庫一路10號,高雄市,楠梓區
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,臺北市,萬華區
...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,臺中市,南區
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,桃園市,龜山區
1266,編緣貓企業社,臺中市南區南門路47號1樓,臺中市,南區
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,臺南市,東區


In [58]:
# 修改欄位名稱
df_filtered = df_filtered.rename(
    columns={
        "legalname": "name",
        "legaladdress": "address",
    }
)
df_filtered

,name,address,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,臺北市,大安區
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,高雄市,左營區
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,高雄市,楠梓區
3,樂沛寵物,高雄市楠梓區土庫一路10號,高雄市,楠梓區
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,臺北市,萬華區
...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,臺中市,南區
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,桃園市,龜山區
1266,編緣貓企業社,臺中市南區南門路47號1樓,臺中市,南區
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,臺南市,東區


In [59]:
# 透過google api並傳送名稱與地址取得place_id
API_KEY = os.getenv("GOOGLE_MAP_KEY_CHGWYELLOW")
result = []

# enumerate會自動將被iterate的物件附上index
for i, (idx, row) in enumerate(df_filtered.iterrows()):
    query = f"{row['name']} {row['address']}"

    result.append(gm.get_place_id(API_KEY, query))
df_filtered["place_id"] = np.nan
df_filtered.loc[:, "place_id"] = result

df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_3904\3790517035.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['ChIJF5gC5M2rQjQRNVDYYkI6w38', 'ChIJt72w85gFbjQRvtDmqmEmqXA', 'ChIJI4yoCi0QbjQRaa3rLmwX9_E', 'ChIJd32-tW4PbjQRc_RTEEXslr8', 'ChIJoaLxpK-pQjQR2IgOJV444ro', 'ChIJ5Y3o1WsfaDQRhfp-uecJY7M', None, 'ChIJ-YWMARE-aTQR6NH9Hn7a7Ag', 'ChIJQSb-nP6lQjQRp25gPneo5Uc', 'ChIJjZwMA5o9aTQRmY3_WnL4jaQ', 'ChIJJ9uDquapQjQRhyjrOg-uwew', 'ChIJcXJcgHSpQjQRNDlpI-2FZdI', 'ChIJi_njB3agQjQRbkQpnS_gTsU', 'ChIJSy6dQtd1bjQR-y7zxoYc6zA', 'ChIJo035d0GpQjQRxOnY9hF0O4k', 'ChIJpfglnWcDaDQROqnauibQ4pE', 'ChIJF6U6RwCpQjQR6vUtNAzXgQ8', 'ChIJ13VK7b0BaDQRrN1sAt5dXoc', 'ChIJTUliRwCFbjQRK2KpSduhEO8', 'ChIJUxVIw-apQjQRdGzFO9XnFKw', 'ChIJWde5Zsvni2MRU7nv4bWTcgM', 'ChIJxy3cuuepQjQR0DPQ2NeCtMo', 'ChIJHcIvT6ypQjQRc9gESCt3ZSY', 'ChIJ_0MqQfN3bjQRMjpqe8pRsx8', 'ChIJaXCe0YGoQjQR-VO0CapX8PI', 'ChIJ6fIImR8faDQRzCnb_YE-ouw', 'ChIJqYpX9X0XaTQRpRiSE6nwFbg', 'C

,name,address,city,district,place_id
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,臺北市,大安區,ChIJF5gC5M2rQjQRNVDYYkI6w38
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,高雄市,左營區,ChIJt72w85gFbjQRvtDmqmEmqXA
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,高雄市,楠梓區,ChIJI4yoCi0QbjQRaa3rLmwX9_E
3,樂沛寵物,高雄市楠梓區土庫一路10號,高雄市,楠梓區,ChIJd32-tW4PbjQRc_RTEEXslr8
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,臺北市,萬華區,ChIJoaLxpK-pQjQR2IgOJV444ro
...,...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,臺中市,南區,ChIJk4sZr_c9aTQRuqpS93LGEF8
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,桃園市,龜山區,ChIJres4rvYeaDQRg9QzSaBXUmo
1266,編緣貓企業社,臺中市南區南門路47號1樓,臺中市,南區,ChIJFyHszAY9aTQRDwDp7dVRn5o
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,臺南市,東區,ChIJNeOGh7l2bjQRotcUscNuI2c


In [82]:
df_filtered = df_filtered.dropna(subset="place_id")

In [61]:
# 透過place_id找到詳細資料
result = []
for _, row in df_filtered.iterrows():
    result.append(gm.gmap_info(row["name"], API_KEY, row["place_id"]))
df_checked = pd.DataFrame(result)
df_checked

,name,place_id,business_status,address,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,貓悅良犬寵物坊,ChIJF5gC5M2rQjQRNVDYYkI6w38,OPERATIONAL,106台灣臺北市大安區基隆路二段110號1樓,0227326189,"[星期一: 13:30 – 22:30, 星期二: 13:30 – 22:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,39.0,121.556998,25.029194,https://maps.google.com/?cid=9206266120076021813,None,2025-09-19
1,東森寵物 左營左大店,ChIJt72w85gFbjQRvtDmqmEmqXA,OPERATIONAL,813台灣高雄市左營區左營大路161號,075818668,"[星期一: 10:00 – 21:00, 星期二: 10:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,198.0,120.289283,22.684462,https://maps.google.com/?cid=8118062004232966334,https://www.etipets.com/,2024-10-20
2,東森寵物 楠梓興楠店,ChIJI4yoCi0QbjQRaa3rLmwX9_E,OPERATIONAL,811台灣高雄市楠梓區興楠路60號,073542228,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,633.0,120.338369,22.731786,https://maps.google.com/?cid=17435430235799268713,https://www.etipets.com/,2023-10-13
3,LaiDua寵物美容＆寵物住宿（安親、貓咪寄宿、高雄寵物住宿）,ChIJd32-tW4PbjQRc_RTEEXslr8,OPERATIONAL,811台灣高雄市楠梓區土庫一路10號,0971757031,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,57.0,120.332154,22.735754,https://maps.google.com/?cid=13805481489121670259,https://www.facebook.com/laiduapetspahotel,2023-11-05
4,犬的造型寵物店,ChIJoaLxpK-pQjQR2IgOJV444ro,OPERATIONAL,108台灣臺北市萬華區艋舺大道172號,0223065696,"[星期一: 11:00 – 17:00, 星期二: 11:00 – 17:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,10.0,121.500846,25.032781,https://maps.google.com/?cid=13466387812790995160,None,2019-12-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,小毛球寵物坊,ChIJk4sZr_c9aTQRuqpS93LGEF8,OPERATIONAL,402台灣臺中市南區國光路443號,0919399621,"[星期一: 13:00 – 21:00, 星期二: 13:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,6.0,120.679761,24.118867,https://maps.google.com/?cid=6850193230308485818,https://www.facebook.com/furballspets/,2025-03-07
1265,東森寵物 桃園巨蛋店,ChIJres4rvYeaDQRg9QzSaBXUmo,OPERATIONAL,333台灣桃園市龜山區萬壽路二段1379號,033490666,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,1962.0,121.325233,24.992488,https://maps.google.com/?cid=7661282262045152387,http://www.104pet.com.tw/,2025-08-13
1266,良辰吉喵貓旅館(全預約制),ChIJFyHszAY9aTQRDwDp7dVRn5o,OPERATIONAL,402台灣臺中市南區南門路47號,0423210631,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,11.0,120.681542,24.122188,https://maps.google.com/?cid=11141713982399250447,https://www.facebook.com/thetimewithcats/,2019-02-04
1267,貓の旅坊 （預約制） 純貓咪旅館及低壓貓洗浴沙龍,ChIJNeOGh7l2bjQRotcUscNuI2c,OPERATIONAL,701台灣臺南市東區裕農路691號,0905966368,"[星期一: 10:00 – 19:30, 星期二: 10:00 – 19:30, 星期三: ...","[establishment, point_of_interest]",5.0,107.0,120.237445,22.988941,https://maps.google.com/?cid=7431905596860061602,https://www.facebook.com/%E8%B2%93%E3%81%AE%E6...,2025-08-20


In [83]:
df_checked = df_checked.dropna(subset="place_id")
df_checked

,name,place_id,business_status,address,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,貓悅良犬寵物坊,ChIJF5gC5M2rQjQRNVDYYkI6w38,OPERATIONAL,106台灣臺北市大安區基隆路二段110號1樓,0227326189,"[星期一: 13:30 – 22:30, 星期二: 13:30 – 22:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,39.0,121.556998,25.029194,https://maps.google.com/?cid=9206266120076021813,None,2025-09-19
1,東森寵物 左營左大店,ChIJt72w85gFbjQRvtDmqmEmqXA,OPERATIONAL,813台灣高雄市左營區左營大路161號,075818668,"[星期一: 10:00 – 21:00, 星期二: 10:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,198.0,120.289283,22.684462,https://maps.google.com/?cid=8118062004232966334,https://www.etipets.com/,2024-10-20
2,東森寵物 楠梓興楠店,ChIJI4yoCi0QbjQRaa3rLmwX9_E,OPERATIONAL,811台灣高雄市楠梓區興楠路60號,073542228,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,633.0,120.338369,22.731786,https://maps.google.com/?cid=17435430235799268713,https://www.etipets.com/,2023-10-13
3,LaiDua寵物美容＆寵物住宿（安親、貓咪寄宿、高雄寵物住宿）,ChIJd32-tW4PbjQRc_RTEEXslr8,OPERATIONAL,811台灣高雄市楠梓區土庫一路10號,0971757031,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,57.0,120.332154,22.735754,https://maps.google.com/?cid=13805481489121670259,https://www.facebook.com/laiduapetspahotel,2023-11-05
4,犬的造型寵物店,ChIJoaLxpK-pQjQR2IgOJV444ro,OPERATIONAL,108台灣臺北市萬華區艋舺大道172號,0223065696,"[星期一: 11:00 – 17:00, 星期二: 11:00 – 17:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,10.0,121.500846,25.032781,https://maps.google.com/?cid=13466387812790995160,None,2019-12-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,小毛球寵物坊,ChIJk4sZr_c9aTQRuqpS93LGEF8,OPERATIONAL,402台灣臺中市南區國光路443號,0919399621,"[星期一: 13:00 – 21:00, 星期二: 13:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,6.0,120.679761,24.118867,https://maps.google.com/?cid=6850193230308485818,https://www.facebook.com/furballspets/,2025-03-07
1265,東森寵物 桃園巨蛋店,ChIJres4rvYeaDQRg9QzSaBXUmo,OPERATIONAL,333台灣桃園市龜山區萬壽路二段1379號,033490666,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,1962.0,121.325233,24.992488,https://maps.google.com/?cid=7661282262045152387,http://www.104pet.com.tw/,2025-08-13
1266,良辰吉喵貓旅館(全預約制),ChIJFyHszAY9aTQRDwDp7dVRn5o,OPERATIONAL,402台灣臺中市南區南門路47號,0423210631,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,11.0,120.681542,24.122188,https://maps.google.com/?cid=11141713982399250447,https://www.facebook.com/thetimewithcats/,2019-02-04
1267,貓の旅坊 （預約制） 純貓咪旅館及低壓貓洗浴沙龍,ChIJNeOGh7l2bjQRotcUscNuI2c,OPERATIONAL,701台灣臺南市東區裕農路691號,0905966368,"[星期一: 10:00 – 19:30, 星期二: 10:00 – 19:30, 星期三: ...","[establishment, point_of_interest]",5.0,107.0,120.237445,22.988941,https://maps.google.com/?cid=7431905596860061602,https://www.facebook.com/%E8%B2%93%E3%81%AE%E6...,2025-08-20


In [84]:
df_merged = df_filtered.merge(
    df_checked,
    how="outer",
    left_on=df_filtered["place_id"],
    right_on=df_checked["place_id"],
    suffixes=["_filtered", "_checked"]
)
df_merged = df_merged.drop(columns=["place_id_filtered", "place_id_checked"])
df_merged = df_merged.drop_duplicates(subset="key_0")
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,新北市板橋區合宜路13號1樓,新北市,板橋區,喵屋喵wu,OPERATIONAL,220台灣新北市板橋區合宜一路27號,0922228719,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,臺南市東區小東路364號1樓,臺南市,東區,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,OPERATIONAL,701台灣臺南市東區小東路364號,0968985129,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,臺北市內湖區臺北市內湖區環山路一段63號1樓,臺北市,內湖區,雅雅愛犬專業美容,OPERATIONAL,114台灣臺北市內湖區環山路一段63號,0227986018,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,臺中市東區和平街259號1樓、地下1樓,臺中市,東區,卡地亞犬舍,OPERATIONAL,401台灣臺中市東區和平街259号號,0933402020,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,ChIJzapf0lypQjQRkrdbOa0zsak,九能寵物用品有限公司,臺北市中山區林森北路413號8樓之21,臺北市,中山區,九能寵物用品有限公司,OPERATIONAL,10491台灣中山區林森北路413號,0225682505,"[星期一: 09:00 – 18:00, 星期二: 09:00 – 18:00, 星期三: ...","[establishment, point_of_interest, store]",NaN,NaN,121.525738,25.059887,https://maps.google.com/?cid=12227611282372278162,None,None
3562,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,新北市淡水區淡金路38巷33-20號1樓,新北市,淡水區,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,OPERATIONAL,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3563,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,桃園市八德區桃園市八德區大和里金和路85號1樓,桃園市,八德區,牠說寵物生活館/美容 住宿 安親,OPERATIONAL,334台灣桃園市八德區金和路85號,033636320,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3564,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,高雄市新興區復興一路14之3號,高雄市,新興區,迷妳寵物店,OPERATIONAL,800台灣高雄市新興區復興一路14-3號,072225588,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [85]:
df_merged = df_merged.dropna(subset="key_0")
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,新北市板橋區合宜路13號1樓,新北市,板橋區,喵屋喵wu,OPERATIONAL,220台灣新北市板橋區合宜一路27號,0922228719,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,臺南市東區小東路364號1樓,臺南市,東區,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,OPERATIONAL,701台灣臺南市東區小東路364號,0968985129,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,臺北市內湖區臺北市內湖區環山路一段63號1樓,臺北市,內湖區,雅雅愛犬專業美容,OPERATIONAL,114台灣臺北市內湖區環山路一段63號,0227986018,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,臺中市東區和平街259號1樓、地下1樓,臺中市,東區,卡地亞犬舍,OPERATIONAL,401台灣臺中市東區和平街259号號,0933402020,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,ChIJzapf0lypQjQRkrdbOa0zsak,九能寵物用品有限公司,臺北市中山區林森北路413號8樓之21,臺北市,中山區,九能寵物用品有限公司,OPERATIONAL,10491台灣中山區林森北路413號,0225682505,"[星期一: 09:00 – 18:00, 星期二: 09:00 – 18:00, 星期三: ...","[establishment, point_of_interest, store]",NaN,NaN,121.525738,25.059887,https://maps.google.com/?cid=12227611282372278162,None,None
3562,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,新北市淡水區淡金路38巷33-20號1樓,新北市,淡水區,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,OPERATIONAL,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3563,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,桃園市八德區桃園市八德區大和里金和路85號1樓,桃園市,八德區,牠說寵物生活館/美容 住宿 安親,OPERATIONAL,334台灣桃園市八德區金和路85號,033636320,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3564,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,高雄市新興區復興一路14之3號,高雄市,新興區,迷妳寵物店,OPERATIONAL,800台灣高雄市新興區復興一路14-3號,072225588,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [86]:
df_merged = df_merged[df_merged["business_status"] == "OPERATIONAL"]
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,新北市板橋區合宜路13號1樓,新北市,板橋區,喵屋喵wu,OPERATIONAL,220台灣新北市板橋區合宜一路27號,0922228719,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,臺南市東區小東路364號1樓,臺南市,東區,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,OPERATIONAL,701台灣臺南市東區小東路364號,0968985129,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,臺北市內湖區臺北市內湖區環山路一段63號1樓,臺北市,內湖區,雅雅愛犬專業美容,OPERATIONAL,114台灣臺北市內湖區環山路一段63號,0227986018,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,臺中市東區和平街259號1樓、地下1樓,臺中市,東區,卡地亞犬舍,OPERATIONAL,401台灣臺中市東區和平街259号號,0933402020,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,ChIJzapf0lypQjQRkrdbOa0zsak,九能寵物用品有限公司,臺北市中山區林森北路413號8樓之21,臺北市,中山區,九能寵物用品有限公司,OPERATIONAL,10491台灣中山區林森北路413號,0225682505,"[星期一: 09:00 – 18:00, 星期二: 09:00 – 18:00, 星期三: ...","[establishment, point_of_interest, store]",NaN,NaN,121.525738,25.059887,https://maps.google.com/?cid=12227611282372278162,None,None
3562,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,新北市淡水區淡金路38巷33-20號1樓,新北市,淡水區,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,OPERATIONAL,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3563,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,桃園市八德區桃園市八德區大和里金和路85號1樓,桃園市,八德區,牠說寵物生活館/美容 住宿 安親,OPERATIONAL,334台灣桃園市八德區金和路85號,033636320,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3564,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,高雄市新興區復興一路14之3號,高雄市,新興區,迷妳寵物店,OPERATIONAL,800台灣高雄市新興區復興一路14-3號,072225588,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [87]:
# 檢查是否有空值
df_merged.isna().any()

key_0               False
name_filtered       False
address_filtered    False
city                False
district            False
name_checked        False
business_status     False
address_checked     False
phone                True
opening_hours        True
types               False
rating               True
rating_total         True
longitude           False
latitude            False
map_url             False
website              True
newest_review        True
dtype: bool

In [90]:
revised_columns = [
    "key_0",
    "name_filtered",
    "name_checked",
    "address_filtered",
    "address_checked",
    "phone",
    "city",
    "district",
    "business_status",
    "opening_hours",
    "types",
    "rating",
    "rating_total",
    "longitude",
    "latitude",
    "map_url",
    "website",
    "newest_review",
]
df_merged = df_merged[revised_columns]
df_merged

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,喵屋喵wu,新北市板橋區合宜路13號1樓,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,臺南市東區小東路364號1樓,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,雅雅愛犬專業美容,臺北市內湖區臺北市內湖區環山路一段63號1樓,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,卡地亞犬舍,臺中市東區和平街259號1樓、地下1樓,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,OPERATIONAL,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,ChIJzapf0lypQjQRkrdbOa0zsak,九能寵物用品有限公司,九能寵物用品有限公司,臺北市中山區林森北路413號8樓之21,10491台灣中山區林森北路413號,0225682505,臺北市,中山區,OPERATIONAL,"[星期一: 09:00 – 18:00, 星期二: 09:00 – 18:00, 星期三: ...","[establishment, point_of_interest, store]",NaN,NaN,121.525738,25.059887,https://maps.google.com/?cid=12227611282372278162,None,None
3562,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,新北市淡水區淡金路38巷33-20號1樓,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3563,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,牠說寵物生活館/美容 住宿 安親,桃園市八德區桃園市八德區大和里金和路85號1樓,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3564,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,迷妳寵物店,高雄市新興區復興一路14之3號,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [101]:
if df_merged["key_0"].duplicated().any():
    df_merged = df_merged.drop_duplicates(subset="key_0")
df_merged

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,喵屋喵wu,新北市板橋區合宜路13號1樓,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,臺南市東區小東路364號1樓,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,雅雅愛犬專業美容,臺北市內湖區臺北市內湖區環山路一段63號1樓,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,卡地亞犬舍,臺中市東區和平街259號1樓、地下1樓,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,OPERATIONAL,0,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,ChIJzapf0lypQjQRkrdbOa0zsak,九能寵物用品有限公司,九能寵物用品有限公司,臺北市中山區林森北路413號8樓之21,10491台灣中山區林森北路413號,0225682505,臺北市,中山區,OPERATIONAL,"[星期一: 09:00 – 18:00, 星期二: 09:00 – 18:00, 星期三: ...","[establishment, point_of_interest, store]",0.0,0.0,121.525738,25.059887,https://maps.google.com/?cid=12227611282372278162,None,0
3562,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,新北市淡水區淡金路38巷33-20號1樓,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3563,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,牠說寵物生活館/美容 住宿 安親,桃園市八德區桃園市八德區大和里金和路85號1樓,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3564,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,迷妳寵物店,高雄市新興區復興一路14之3號,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [92]:
# 計算欄位空值補0
fillna_columns = ["opening_hours", "rating", "rating_total", "newest_review"]
df_merged[fillna_columns] = df_merged[fillna_columns].fillna(0)
df_merged.isna().any()

key_0               False
name_filtered       False
name_checked        False
address_filtered    False
address_checked     False
phone                True
city                False
district            False
business_status     False
opening_hours       False
types               False
rating              False
rating_total        False
longitude           False
latitude            False
map_url             False
website              True
newest_review       False
dtype: bool

In [93]:
# 連線資料庫
host = os.getenv("MYSQL_IP")
port = int(os.getenv("MYSQL_PORTT"))
user = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
db = os.getenv("MYSQL_DB_NAME")
conn, cursor = connDB.connect_db(host, port, user, password, db)

✅ TJR103_1資料庫已成功連線


In [99]:
# 讀取location表格的資料
sql = """
select loc_id, city, district
from location;
"""
cursor.execute(sql)
loc = cursor.fetchall()

df_loc = pd.DataFrame(data=loc, columns=["loc_id", "city", "district"])
df_loc

,loc_id,city,district
0,NTP001,新北市,新北市
1,NTP002,新北市,板橋區
2,NTP003,新北市,三重區
3,NTP004,新北市,中和區
4,NTP005,新北市,永和區
...,...,...,...
154,KSH034,高雄市,甲仙區
155,KSH035,高雄市,杉林區
156,KSH036,高雄市,內門區
157,KSH037,高雄市,茂林區


In [106]:
# merge df_merged和df_loc
df_final = df_merged.merge(df_loc, left_on=["city", "district"], right_on=["city", "district"], how="left")
df_final

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review,loc_id
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,喵屋喵wu,新北市板橋區合宜路13號1樓,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29,NTP002
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,臺南市東區小東路364號1樓,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18,TNA033
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,雅雅愛犬專業美容,臺北市內湖區臺北市內湖區環山路一段63號1樓,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31,TPE011
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,卡地亞犬舍,臺中市東區和平街259號1樓、地下1樓,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,OPERATIONAL,0,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21,TCH003
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09,KSH006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,ChIJzapf0lypQjQRkrdbOa0zsak,九能寵物用品有限公司,九能寵物用品有限公司,臺北市中山區林森北路413號8樓之21,10491台灣中山區林森北路413號,0225682505,臺北市,中山區,OPERATIONAL,"[星期一: 09:00 – 18:00, 星期二: 09:00 – 18:00, 星期三: ...","[establishment, point_of_interest, store]",0.0,0.0,121.525738,25.059887,https://maps.google.com/?cid=12227611282372278162,None,0,TPE005
1109,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,新北市淡水區淡金路38巷33-20號1樓,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07,NTP011
1110,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,牠說寵物生活館/美容 住宿 安親,桃園市八德區桃園市八德區大和里金和路85號1樓,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28,TYN009
1111,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,迷妳寵物店,高雄市新興區復興一路14之3號,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09,KSH007


In [109]:
df_final[df_final["loc_id"].isna()]

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review,loc_id
81,ChIJ37BeQ_KvQjQRsdFJ5N4lZcQ,愛玩犬寵美屋,愛玩犬寵物美容寵物住宿,臺北市北投區中正街69號1樓及2樓,112台灣台北市北投区中正街69號1樓,0228928100,臺北市,北投區,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.6,44.0,121.499814,25.132334,https://maps.google.com/?cid=14151759043324662193,None,2025-07-19,NaN
99,ChIJ4RnvY4uuQjQRPS4RbFsrMz0,辛普森寵物概念館,辛普森寵物概念館,臺北市北投區義理街64號1樓,112台灣臺北市北投區義理街64號,0228200011,臺北市,北投區,OPERATIONAL,"[星期一: 10:30 – 20:00, 星期二: 10:30 – 20:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.6,14.0,121.515999,25.117598,https://maps.google.com/?cid=4409916131780931133,None,2021-05-18,NaN
194,ChIJAyDXDEevQjQR2t9h6OoeU58,喵了個汪寵物生活館,喵了個汪寵物美容寵物住宿,臺北市北投區中和街275號1樓及2樓,112台灣臺北市北投區中和街275號,None,臺北市,北投區,OPERATIONAL,"[星期一: 休息, 星期二: 休息, 星期三: 11:00 – 20:00, 星期四: 11...","[establishment, pet_store, point_of_interest, ...",5.0,80.0,121.500439,25.140198,https://maps.google.com/?cid=11480553869367435226,https://www.meowawoof2023.com/,2025-04-02,NaN
196,ChIJB2yU8X12bjQRnZUuCSAu2RY,噗荳寵物生活館,噗荳寵物生活館,臺南市中西區南門路179之2號1樓,700台灣臺南市中西區南門路179-2號,062156586,臺南市,中西區,OPERATIONAL,"[星期一: 11:00 – 21:00, 星期二: 11:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,6.0,120.204363,22.986007,https://maps.google.com/?cid=1646397853908506013,None,2021-01-08,NaN
228,ChIJCYcQ_Np1bjQRiWGE6UIXrjs,凱凱寵物生活館,凱凱寵物館,臺南市中西區中西區西門路二段39號,700台灣臺南市中西區西門路二段39號,None,臺南市,中西區,OPERATIONAL,0,"[establishment, pet_store, point_of_interest, ...",5.0,1.0,120.197855,22.991067,https://maps.google.com/?cid=4300400270338449801,None,2024-08-01,NaN
288,ChIJFWWi-JKuQjQRSTEf1teKOoY,順竩有限公司,順達有限公司,臺北市北投區復興四路16之1號1樓,112台灣臺北市北投區致遠一路一段46巷20號,0228237215,臺北市,北投區,OPERATIONAL,0,"[establishment, point_of_interest]",4.0,1.0,121.516990,25.109555,https://maps.google.com/?cid=9672195809346531657,None,2019-08-22,NaN
337,ChIJHzwWxG52bjQRrtEQdb-R2f0,愛諾寵物生活館,愛諾寵物生活館(臨安館) ｜中秋、國慶&光復連假有營業！｜每週四店休,臺南市中西區臨安路一段16號,70060台灣臺南市中西區臨安路一段16號,062263666,臺南市,中西區,OPERATIONAL,"[星期一: 10:00 – 20:00, 星期二: 10:00 – 20:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.3,64.0,120.189558,22.997888,https://maps.google.com/?cid=18291811613030797742,https://www.iknowpetshop.com/,2022-07-23,NaN
399,ChIJM1pTJnZ3bjQROERXd6514_c,小犬座寵物旅館,小犬座寵物旅館Canis Minor,臺南市中西區海安路一段30號,700台灣臺南市中西區海安路一段30號,0931999713,臺南市,中西區,OPERATIONAL,"[星期一: 10:00 – 21:00, 星期二: 10:00 – 21:00, 星期三: ...","[establishment, point_of_interest]",5.0,15.0,120.195108,22.988164,https://maps.google.com/?cid=17862249939268486200,None,2025-08-11,NaN
452,ChIJOz3BXFp3bjQRjaKbBXn5HlY,米雅寵物工作室,米雅寵物工作室,臺南市中西區友愛街359號,700台灣臺南市中西區友愛街359號,None,臺南市,中西區,OPERATIONAL,0,"[establishment, point_of_interest]",5.0,2.0,120.194039,22.992525,https://maps.google.com/?cid=6205671634743566989,None,2023-10-29,NaN
563,ChIJVYkgw4ysQjQR0rFoI1Lf-Tg,黑鼻子寵物服務有限公司,黑鼻子寵物旅館 BlackNose Pet Hotel,臺北市北投區自強街164號1樓,112059台灣臺北市北投區自強街164號1樓,0287912541,臺北市,北投區,OPERATIONAL,"[星期一: 08:30 – 21:00, 星期二: 08:30 – 21:00, 星期三: ...","[establishment, point_of_interest]",4.4,93.0,121.516886,25.111020,https://maps.google.com/?cid=4105558079199293906,https://www.blacknosepethotel.com/,2024-05-16,NaN


In [ ]:
df_final = df_final.drop(columns=["city_y"])
df_final

In [ ]:
df_final = df_final.rename(columns={"city_x": "city"})
df_final

In [ ]:
columns = [
    "key_0",
    "name_checked",
    "address_checked",
    "phone",
    "city",
    "district",
    "business_status",
    "opening_hours",
    "types",
    "rating",
    "rating_total",
    "longitude",
    "latitude",
    "map_url",
    "website",
    "newest_review",
]

df_final = df_final[columns]
df_final

In [20]:
# 儲存ETL後CSV檔
path = "data/processed/pet_hotel_ETL.csv"
sd.store_to_csv_no_index(df_merged, path)

[✓] 不含index的CSV檔已存檔完畢
